In [1]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# statmodels
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima_model import ARMA, ARIMA
# datetime
from datetime import datetime

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_rows', None)



In [2]:
# load the data
df = pd.read_csv('us_airline_carrier_passenger.csv')
df.head()

,month,passengers
0,2000-01-01,46492
1,2000-02-01,48526
2,2000-03-01,58764
3,2000-04-01,56033
4,2000-05-01,58201


In [3]:
# creating predictors

exog_df=df.copy()

exog_df['month']=exog_df['month'].astype('datetime64[ns]')

# month predictors

exog_df['month']=exog_df['month'].dt.month
month_dummies=pd.get_dummies(exog_df['month'], prefix='Month')

# drop one month predictor so hessian is invertible

month_dummies=month_dummies[[x for x in month_dummies.columns if x!='Month_12']]

month_dummies.head()




,Month_1,Month_2,Month_3,Month_4,Month_5,Month_6,Month_7,Month_8,Month_9,Month_10,Month_11
0,1,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,0


In [28]:
# creating data to test evaluation pipeline with

eval_test_df=pd.concat([df[['passengers']],month_dummies],axis=1)

eval_test_df.columns


# rolling forecast function for determining p,d,q values:

def best_pdq(df, 
endog_col, 
exog_cols,
train_length, 
forecast_length,
start_p=2, 
trend='c',                             
start_q=2, 
max_p=5, 
max_d=2, 
max_q=5,
d=None,
seasonal=None,
stationary=None,
trace=None,
scoring =None):


    # required functions
    # deal with negative values for boxcox
    def non_zero(data_df):

        data_df2=data_df.copy()

        data_df2=data_df2[~data_df2.isnull()]


        if data_df2.min()[0]<0:

            fill_df=data_df2.copy()

            positive_val=abs(fill_df.min()[0]*1.0001)

        else:
            fill_df=data_df2.copy()

            positive_val=0

            positive_df=fill_df+positive_val

        return positive_df, positive_val




    forecast_list = []
    test_list = []
    i = 0
    while i <= len(df) - train_length - forecast_length:

        try:
            # filter df for training window length + forecast length
            df_window = df.iloc[i:i+train_length+forecast_length]

            # split into train and forecast dataframes
            train_df = df_window.iloc[:train_length]
            forecast_df = df_window.iloc[train_length:]

            # split train and forecast dataframes into endog and exog dataframes
            train_endog = train_df[[endog_col]]
            train_exog = train_df[exog_cols]
            forecast_exog = forecast_df[exog_cols]

            # determining best model

            df1,pos_val=non_zero(data_df=train_endog.copy())



            # apply boxcox transform

            import scipy.stats as stats


            df1['boxcox'], param = stats.boxcox(df1[df1.columns[0]])
            

            from pmdarima import auto_arima

            auto_model = auto_arima(df1['boxcox'],
                                    exogeneous=train_exog, 
                                    start_p=start_p, 
                                    d=d, 
                                    start_q=start_q, 
                                    max_p=max_p, 
                                    max_d=max_d, 
                                    max_q=max_q,
                                    seasonal=seasonal,
                                    stationary=stationary,
                                    trace=trace,
                                    scoring ='mse',
                                   trend=trend).order



            # append forecast and actual values to lists
            forecast_list.extend([auto_model])
    

            i += 1

            
        except Exception as err:

            print('error:')
            print(err)
            break

    # averaging and rounding p,d,q values

    p_list=[x[0] for x in forecast_list]
    d_list=[x[1] for x in forecast_list]
    q_list=[x[2] for x in forecast_list]

    estimated_model=[round(np.mean(p_list)),round(np.mean(d_list)),round(np.mean(q_list))]

    return forecast_list, estimated_model


# In[18]:

print('stable p,d,q values:')
print('')

best_pdq(df=eval_test_df, 
endog_col='passengers', 
exog_cols=['Month_1', 'Month_2', 'Month_3', 'Month_4', 'Month_5',
       'Month_6', 'Month_7', 'Month_8', 'Month_9', 'Month_10', 'Month_11'],
train_length=24, 
forecast_length=4,
start_p=2, 
trend='c',                             
start_q=2, 
max_p=5, 
max_d=2, 
max_q=5,
d=None,
seasonal=None,
stationary=None,
trace=None,
scoring =None)[1]

stable p,d,q values:



[1, 0, 0]

In [26]:
# evaluation pipeline

def model_forecaster(train,
                           exog_train,
                           exog_forecast,
                           steps,
                          start_p=2, 
                           trend='c',                             
                            start_q=2, 
                            max_p=5, 
                            max_d=2, 
                            max_q=5,
                           d=None,
                            seasonal=None,
                            stationary=None,
                            trace=None,
                            scoring =None,
                           model=None
                          ):
    
    # deal with negative values for boxcox
    def non_zero(data_df):
    
        data_df2=data_df.copy()

        data_df2=data_df2[~data_df2.isnull()]


        if data_df2.min()[0]<0:

            fill_df=data_df2.copy()

            positive_val=abs(fill_df.min()[0]*1.0001)

        else:
            fill_df=data_df2.copy()

            positive_val=0

            positive_df=fill_df+positive_val

        return positive_df, positive_val
        
    df,pos_val=non_zero(data_df=train.copy())
    
    
    
    # apply boxcox transform

    import scipy.stats as stats


    df['boxcox'], param = stats.boxcox(df[df.columns[0]])
#     print('Optimal lambda', param)


    # if model is given can skip determining
    if model == None:
        
        # get model order from auto_arima

        from pmdarima import auto_arima

        auto_model = auto_arima(df['boxcox'],
                                exogeneous=exog_train, 
                                start_p=start_p, 
                                d=d, 
                                start_q=start_q, 
                                max_p=max_p, 
                                max_d=max_d, 
                                max_q=max_q,
                                seasonal=seasonal,
                                stationary=stationary,
                                trace=trace,
                                scoring ='mse',
                               trend=trend).order
        
    else:

        auto_model=model





    # just predicting number of steps ahead

    def forecaster(model_order,
                         stationary_train,
                         exog_train,
                         exog_forecast,
                         steps):

        model_arima_best1 = ARIMA(endog=stationary_train,
                                  exog=exog_train,
                                  order=model_order).fit()



        # make prediction
        prediction_arima_best1 = model_arima_best1.forecast(steps=steps,
                                                           exog=exog_forecast)[0]


               
        forecasted_values_arima_best1 = prediction_arima_best1

        return forecasted_values_arima_best1


    
    


    test_forecast=forecaster(
        model_order=auto_model,
    stationary_train=df['boxcox'],
        exog_train=exog_train,
        exog_forecast=exog_forecast,
    steps=steps
                    )


    # applying inverse boxcox transformation

    from scipy.special import inv_boxcox
    
    positive_reverse_box_cox=[x-pos_val for x in inv_boxcox(test_forecast, param)]
    

    return positive_reverse_box_cox


# rolling forecast evaluation pipeline

def evaluate_model(df, endog_col, exog_cols, train_length, forecast_length,model=None):
    forecast_list = []
    test_list = []
    i = 0
    while i <= len(df) - train_length - forecast_length:
        
        try:
            # filter df for training window length + forecast length
            df_window = df.iloc[i:i+train_length+forecast_length]

            # split into train and forecast dataframes
            train_df = df_window.iloc[:train_length]
            forecast_df = df_window.iloc[train_length:]

            # split train and forecast dataframes into endog and exog dataframes
            train_endog = train_df[[endog_col]]
            train_exog = train_df[exog_cols]
            forecast_exog = forecast_df[exog_cols]

            # forecast using arima_forecast_model
            forecast = model_forecaster(train=train_endog, 
                                              exog_train=train_exog, 
                                              exog_forecast=forecast_exog, 
                                              steps=forecast_length,
                                             model=model)

            # append forecast and actual values to lists
            forecast_list.extend(forecast)
            test_list.extend(forecast_df[:forecast_length][endog_col].values.tolist())

            i += 1
            
        except Exception as err:
            
            print('error:')
            print(err)
            break

    return np.array(forecast_list), np.array(test_list)

In [29]:
month_predictors=['Month_1', 'Month_2', 'Month_3', 'Month_4', 'Month_5',
       'Month_6', 'Month_7', 'Month_8', 'Month_9', 'Month_10', 'Month_11']


forecast_list, test_list=evaluate_model(df=eval_test_df, 
                                                       endog_col='passengers', 
                                                       exog_cols=month_predictors, 
                                                       train_length=24, 
                                                       forecast_length=4,
                                                      model=(1,0,0))



print('mape: ')
np.mean([abs((forecast_list[i]-test_list[i])/test_list[i]) for i in range(len(test_list))])

mape: 


0.0355455642028968